## Hagamos calculos en tiempo real

Primero, importemos las librerias que nos interesa.

In [ ]:
import pandas as pd
import numpy as np

Luego llega el momento de empezar a explorar el conjunto de datos que queremos trabajar:

* Primero debemos cargar el documento fuente en memoria.
* Segundo, exploremos una pequeña muestra del conjunto de datos y miremos como se ven.

In [ ]:
df = pd.read_excel("sources/libro_auxiliar.xlsx")
df.head()

Se aprecia que realmente el conjunto de datos tiene problemas, el principal parece estar relacionado con el hecho del formato de la fuente, corrijamoslo:

* Primero, cargamos nuevamente el documento en un DataFrame de Pandas, pero en esta ocasion saltamos las filas de datos vacias.
* Segundo, demos una mirada mas profunda a todo el conjunto.

In [ ]:
df = pd.read_excel("sources/libro_auxiliar.xlsx", skiprows=2)
df

Evidentemente podemos limpiar (*scrubingun* es el termino preferido) un poco mas los datos.

* Primero, identificamos filas y columnas de datos vacios, que introducen ruido o que son innecesarias y los descartamos.
* Segundo, modificamos la estructura de datos para que tenga mejor lectura y se manipulen mejor los datos.
* Tercero, volvamos a mirar.

In [ ]:
# Comando para descartar filas con datos de tipo Null
df = df[df.Fecha.notnull()]

# Comando para descartar filas que contienen un "string" especifico.
df[df.NomCuenta.str.contains("Total") == False]

# Comando para descartar columnas dentro del conjunto de datos.
df.drop(['ChequeNumero', 'Doc'], axis=1, inplace=True)

# Cambiamos los nombres de algunas de las columnas.
df.rename(inplace=True, columns = {
                                    'NomCuenta':'cuenta_detallada', 
                                    'Tercero':'tercero',
                                    'Notas':'detalle', 'Débitos':'debitos', 
                                    'Créditos':'creditos', 'Saldo':'saldo'
                                })

# Creamos un indice pivotable a partir de una columna.
df['mes'] = pd.DatetimeIndex(df['Fecha']).month

# Rellenamos
df.cuenta_detallada.fillna(method='ffill', inplace=True)
df['clase'] = df.cuenta_detallada.str[:1]
df['grupo'] = df.cuenta_detallada.str[:2]
df['cuenta'] = df.cuenta_detallada.str[:4]
df['subcuenta'] = df.cuenta_detallada.str[:6]
df['cuenta_especifica'] = df.cuenta_detallada.str[:8]
df['nombre_cuenta'] = df.cuenta_detallada.str[7:]
df['consolidado'] = df.creditos - df.debitos
#df = df[df.mes != 12] # This line drops december of previous year, but careful because it can drop actual december 
#df.index = df.mes
df = df[['clase', 'grupo', 'cuenta', 'subcuenta', 'cuenta_especifica', 'nombre_cuenta', 'cuenta_detallada',
         'tercero', 'Fecha', 'detalle', 'debitos', 'creditos', 'saldo', 'consolidado', 'mes']]
df.head()

Ahora hagamos algo aun mas chevere, reemplacemos Excel.

Creemos una *pivot table* de Pandas (es igual a una tabla dinamica... en Python...

**BUUUUUUMMMMM** (La cabeza explota)

In [ ]:
# Definimos la estructura del queryset a manejar.
query = 'cuenta == ['
cuentas = (4105, 4110, 4115, 4120, 4125, 4130, 4135, 4140, 4145, 4150, 4155, 4160, 4165,
           4170, 4175, 4205, 4210, 4215, 4220, 4225, 4230, 4235, 4240, 4245, 4248, 4250,
           4255, 4260, 4265, 4275, 4295, 4705, 5105, 5110, 5115, 5120, 5125, 5130, 5135,
           5140, 5145, 5150, 5155, 5160, 5165, 5195, 5199, 5205, 5210, 5215, 5220, 5225,
           5230, 5235, 5240, 5245, 5250, 5255, 5260, 5265, 5270, 5295, 5299, 5305, 5310,
           5315, 5395, 5405, 5905, 6105, 6110, 6115, 6120, 6125, 6130, 6135, 6140, 6145,
           6150, 6155, 6160, 6165, 6170, 6205, 6210, 6215, 6220, 6225, 7105, 7110, 7115,
           7120, 7125, 7130, 7135, 7140, 7145, 7150, 7155, 7160, 7165, 7170, 7175, 7180,
           7185, 7190, 7195, 7200, 7205, 7210, 7215, 7220, 7225, 7230, 7235, 7240, 7245,
           7250, 7255, 7260, 7265, 7270, 7275, 7280, 7285, 7290, 7295, 7300, 7305, 7310,
           7315, 7320, 7325, 7330, 7335, 7340, 7345, 7350, 7355, 7360, 7365, 7370, 7375,
           7380, 7385, 7390, 7395)

indice = []
for i in cuentas:
    indice.append(str(i))

for i in cuentas:
    i = '"' + str(i) + '", '
    query = query + i

query = query + ']'
tb = pd.pivot_table(df, index=['cuenta', 'nombre_cuenta'], values=['consolidado'], 
                    columns=['mes'], aggfunc=np.sum, fill_value=0).query(query)

# Y veamos como queda esta cosa.
tb